<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/logo-bdc.png" align="right" width="64"/>

# <span style="color: #336699">Land use and land cover classification in the Brazilian Cerrado biome using Brazil Data Cube</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Rolf E. O. Simões <sup><a href="mailto:rolf.simoes@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0003-0953-4132"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Alber H. Sanchez <sup><a href="mailto:alber.ipia@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0001-7966-2880"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Felipe M. Carlos <sup><a href="mailto:felipe.carlos@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0002-3334-4315"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Leonardo S. Vieira <sup><a href="mailto:leonardo.vieira@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0002-3397-6232"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>,<br/>
    Karine R. Ferreira <sup><a href="mailto:karine.ferreira@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0003-2656-5504"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Lubia Vinhas <sup><a href="mailto:lubia.vinhas@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0003-1104-3607"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup>* <a href="mailto:gilberto.queiroz@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    <sup>*</sup> Author to whom correspondence should be addressed.
    <br/><br/>
    February 24, 2021
</div>

<br/>

<div style="text-align: justify;  margin-left: 10%; margin-right: 10%;">
<b>Abstract.</b> This Jupyter Notebook compendium contains useful information for the creation of land use and land cover (LULC) maps using Earth observations data cubes and machine learning (ML) techniques. The code is based on the research pipeline described in the paper <em>Earth Observation Data Cubes for Brazil: Requirements, Methodology and Products</em>. These notebooks access open data available in the Brazil Data Cube platform.
</div>    

<br/>
<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is supplement to the <a href="https://www.mdpi.com/2072-4292/12/24/4033/htm#sec5-remotesensing-12-04033" target="_blank">Section 5</a> of the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    Ferreira, K.R.; Queiroz, G.R.; Vinhas, L.; Marujo, R.F.B.; Simoes, R.E.O.; Picoli, M.C.A.; Camara, G.; Cartaxo, R.; Gomes, V.C.F.; Santos, L.A.; Sanchez, A.H.; Arcanjo, J.S.; Fronza, J.G.; Noronha, C.A.; Costa, R.W.; Zaglia, M.C.; Zioti, F.; Korting, T.S.; Soares, A.R.; Chaves, M.E.D.; Fonseca, L.M.G. 2020. Earth Observation Data Cubes for Brazil: Requirements, Methodology and Products. Remote Sens. 12, no. 24: 4033. DOI: <a href="https://doi.org/10.3390/rs12244033" target="_blank">10.3390/rs12244033</a>.
    </div>
</div>

# <span style="color: #336699">Validation results for CBERS-4/AWFI classification</span>
<hr style="border:1px solid #0077b9;">

This document presents the process for the validation of the results generated in the classification made using the CBERS-4/AWFI data cube. 

The validation procedure of the land use and land cover thematic followed the good practice guidelines for accuracy assessment based on reference data proposed by [Olofsson et al. (2014)](https://doi.org/10.1016/j.rse.2014.02.015).

> The Jupyter Notebook entitled [Land Use and Cover Mapping from CBERS-4/AWFI Data Cubes](./02_CB4_64_16D_STK-1.ipynb) describes in detail how to create the thematic map used here.

## <span style="color: #336699">Loading the software packages</span>
<hr style="border:0.5px solid #0077b9;">

In this example, the validation process uses the [SITS R package](https://github.com/e-sensing/sits):

In [ ]:
library(sits)

We are going to use functionalities from the `sf` package as well:

In [ ]:
library(sf)

## <span style="color: #336699">Load data</span>
<hr style="border:0.5px solid #0077b9;">

To perform the validation, it is initially necessary to load the classification results and the samples used as a reference.

In [ ]:
classification_results <- raster::raster(paste0("/vsicurl/https://brazildatacube.dpi.inpe.br/",
                                                "geo-knowledge-hub/bdc-article/results/CB4_64_16D_STK_1/",
                                                "cube_to_classify_022024_probs_class_2018_8_2019_7_v1.tif"))

validation_samples <- readRDS(url(paste0("https://brazildatacube.dpi.inpe.br/",
                                         "geo-knowledge-hub/bdc-article/validation-samples/",
                                         "validation-samples.rds")))

> If you want to use the output generated in the previous Jupyter Notebook, entitled [Land Use and Cover Mapping from CBERS-4/AWFI Data Cubes](./02_CB4_64_16D_STK-1.ipynb), you can replace the directory `results/CB4_64_16D_STK_1/` by the path `~/work/bdc-article/results/CB4_64_16D_STK_1` in the first line of the above cell.

The cell code below shows the region of the validation samples:

In [ ]:
library(leaflet)

leaflet::leaflet(validation_samples$geom, width="100%", height="600px") %>%
    addProviderTiles(providers$Esri.WorldImagery) %>%
    setView(lng = -45.5, lat = -12.8, zoom = 12) %>%
    addCircleMarkers(radius = 6, color = 'red', stroke = TRUE, fillOpacity = 0.5)

## <span style="color: #336699">Extract predicted values</span>
<hr style="border:0.5px solid #0077b9;">

With the rasters and reference samples loaded, it will now be necessary to extract the values corresponding to each of the validation samples' locations from the classified raster. This extraction allows the comparison of the predicted and reference values.

> For extraction at each location, the `extract` function of the [raster package](https://cran.r-project.org/web/packages/raster/raster.pdf) is used.

In the next cell $x$ corresponds to the classified raster and $y$ to the location of the validation samples:

In [ ]:
predicted_values <- raster::extract(
  x = classification_results, 
  y = validation_samples$geom
)

The `predicted_values` is a vector with the same length as `validation_samples`, whose values correspond to classes from the input classified raster.

## <span style="color: #336699">Transform extracted values</span>
<hr style="border:0.5px solid #0077b9;">

We base the validation on two classes, the `Natural Vegetation` class and on a new broader class named `Anthropic`. 

To obtain the `Anthropic` class we have to merge the `Crop` and `Pasture` classes. So, in the `predicted_values` vector we will temporarily relabel the cells `Natural Vegetation` from code `2` to `5`:

In [ ]:
predicted_values[predicted_values == 2] <- 5

Then, we can relabel `Pasture` (1) and `Agriculture` (3) to the equivalent PRODES `Anthropic` class (2):

In [ ]:
predicted_values[predicted_values == 1] <- 2
predicted_values[predicted_values == 3] <- 2

Finally, we can set the `Natural Vegetation` cells to value `1` again:

In [ ]:
predicted_values[predicted_values == 5] <- 1

> When the classification process is carried out, there is no way to specify the values assigned to each of the classes being identified. Thus, it may be necessary to check the classification values and their corresponding elements so that the values are used for comparison. 

## <span style="color: #336699">Evaluating</span>
<hr style="border:0.5px solid #0077b9;">

In [ ]:
sits_conf_matrix(list(
  predicted = predicted_values,
  reference = validation_samples$reference
))